In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('MRR raw.csv')

#rename the columns to make referencing easier
df=df.rename(columns={'Search Term (evar18)':'Search Term',
                   'Product Rank (evar12)':'Product Rank',
                   'Product Rank Instance (Instance of evar12)':'Occurences'})

#create a SOP column which product of the product rank for a search term and the number of times the product rank occured
df['SOP']=df['Product Rank']*df['Occurences']

#group the data set by search term so that we can find the average product rank of a search term
dfg=df.groupby('Search Term')

#Sum up all integer columns of the grouped dataset
dfgs=dfg.sum()

#create columns MR and MRR where
#MR= SOP/Sum of Occurences
dfgs['MR']=round(dfgs['SOP']/dfgs['Occurences'],2)
#MRR= Inv of MR
dfgs['MRR']=round((1/dfgs['MR']),2)
#avg= average MRR for a search term
avg=dfgs['MRR'].mean()
print('MRR for the week is :',avg)
dfgs.head
#export the dfgs dataset to a csv
dfgs.to_csv('mrr.csv')

### New approach

## Revised approach to MRR

### Search term wise MRR

In [ ]:
#Read the file
df=pd.read_csv('MRR20220701-20230114.csv')
df.sample(5)

In [ ]:
#create a Reciprocal rank column
df['Reciprocal']=1/df['Product Rank (evar12)']

#calculate search term wise MRR
df_search_term=df.groupby(by='Search Term (evar18)').mean().reset_index()
df_search_term=df_search_term[['Search Term (evar18)','Product Rank (evar12)','Reciprocal']]

#Adding a count column for further analysis
df_search_count=df.groupby(by='Search Term (evar18)').count().reset_index()
df_search_count=df_search_count[['Search Term (evar18)','Reciprocal']]
df_search_count.rename(columns={'Reciprocal':'Count'},inplace=True)
df_search_term=df_search_term.merge(df_search_count,on='Search Term (evar18)')
df_search_term.to_csv('MRR_searchterm.csv')
plt.hist(df_search_term['Reciprocal'],bins=20,label='Distribution of MRR')

In [ ]:
df_search_term=df_search_term[df_search_term['Count']>500]
plt.hist(df_search_term['Reciprocal'],bins=20,label='Distribution of MRR')

In [ ]:
df_search_term.describe()

In [ ]:
df_top_1000=df_search_term.sort_values(by='Count',ascending=False)
df_top_1000=df_top_1000.iloc[0:1000]
df_top_1000=df_search_term.sort_values(by='Reciprocal',ascending=False)
df_top_1000.head(50)

### Monthly MRR for top 1000 search terms(Jul-Jan)

In [ ]:
#Monthly breakdown of top 1000 search terms
import warnings
warnings.filterwarnings("ignore")
#Read the file
df=pd.read_csv('MRR20220701-20230114.csv')

In [ ]:
#create a Reciprocal rank column
df['Reciprocal']=1/df['Product Rank (evar12)']

#Add month column
df['Time Stamp (prop3)']=pd.to_datetime(df['Time Stamp (prop3)'])
df['Month']=df['Time Stamp (prop3)'].dt.month
df=df[df['Month']!=6]

#Group the df by Month+ST such that we get Mean of each search term for the month
df_month_mrr=df.groupby(by=['Month','Search Term (evar18)']).mean().reset_index()
df_month_mrr=df_month_mrr[['Month','Search Term (evar18)','Reciprocal']]

#Group the df by Month+ST such that we get count of each search term for the month
df_month_count=df.groupby(by=['Month','Search Term (evar18)']).count().reset_index()
df_month_count=df_month_count[['Search Term (evar18)','Reciprocal','Month']]
df_month_count.rename(columns={'Reciprocal':'Reciprocal Count'},inplace=True)

#Final dataframe
df_final=df_month_count.merge(df_month_mrr,on=['Month','Search Term (evar18)'])

#create df for each month, sort the df by the count of each sort term, get the first 1000 rows and find the mean of this df
df_Jul=df_final[df_final['Month']==7]
df_Jul=df_Jul.sort_values(by='Reciprocal Count',ascending=False)
df_Jul_1000=df_Jul.iloc[0:1000]
print('MRR for Jul is',df_Jul_1000['Reciprocal'].mean())

df_Aug=df_final[df_final['Month']==8]
df_Aug=df_Aug.sort_values(by='Reciprocal Count',ascending=False)
df_Aug_1000=df_Aug.iloc[0:1000]
print('MRR for Aug is',df_Aug_1000['Reciprocal'].mean())

df_Sep=df_final[df_final['Month']==9]
df_Sep=df_Sep.sort_values(by='Reciprocal Count',ascending=False)
df_Sep_1000=df_Sep.iloc[0:1000]
print('MRR for Sep is',df_Sep_1000['Reciprocal'].mean())


df_Oct=df_final[df_final['Month']==10]
df_Oct=df_Oct.sort_values(by='Reciprocal Count',ascending=False)
df_Oct_1000=df_Oct.iloc[0:1000]
print('MRR for Oct is',df_Oct_1000['Reciprocal'].mean())


df_Nov=df_final[df_final['Month']==11]
df_Nov=df_Nov.sort_values(by='Reciprocal Count',ascending=False)
df_Nov_1000=df_Nov.iloc[0:1000]
print('MRR for Nov is',df_Nov_1000['Reciprocal'].mean())


df_Dec=df_final[df_final['Month']==12]
df_Dec=df_Dec.sort_values(by='Reciprocal Count',ascending=False)
df_Dec_1000=df_Dec.iloc[0:1000]
print('MRR for Dec is',df_Dec_1000['Reciprocal'].mean())

df_Jan=df_final[df_final['Month']==1]
df_Jan=df_Jan.sort_values(by='Reciprocal Count',ascending=False)
df_Jan_1000=df_Jan.iloc[0:1000]
print('MRR for Jan is',df_Jan_1000['Reciprocal'].mean())

In [ ]:
df_Jan_1000.sample(5)

### Device wise split for MRR per month

In [ ]:
df['Mobile Device Type'].value_counts()

In [ ]:
df_mobile=df[df['Mobile Device Type']=='Mobile Phone']
df_laptop=df[df['Mobile Device Type']=='Other']
df_tablet=df[df['Mobile Device Type']=='Tablet']

### Mobile

In [ ]:
df=df_mobile
#create a Reciprocal rank column
df['Reciprocal']=1/df['Product Rank (evar12)']

#Add month column
df['Time Stamp (prop3)']=pd.to_datetime(df['Time Stamp (prop3)'])
df['Month']=df['Time Stamp (prop3)'].dt.month
df=df[df['Month']!=6]

#Group the df by Month+ST such that we get Mean of each search term for the month
df_month_mrr=df.groupby(by=['Month','Search Term (evar18)']).mean().reset_index()
df_month_mrr=df_month_mrr[['Month','Search Term (evar18)','Reciprocal']]

#Group the df by Month+ST such that we get count of each search term for the month
df_month_count=df.groupby(by=['Month','Search Term (evar18)']).count().reset_index()
df_month_count=df_month_count[['Search Term (evar18)','Reciprocal','Month']]
df_month_count.rename(columns={'Reciprocal':'Reciprocal Count'},inplace=True)

#Final dataframe
df_final=df_month_count.merge(df_month_mrr,on=['Month','Search Term (evar18)'])

#create df for each month, sort the df by the count of each sort term, get the first 1000 rows and find the mean of this df
df_Jul=df_final[df_final['Month']==7]
df_Jul=df_Jul.sort_values(by='Reciprocal Count',ascending=False)
df_Jul_1000=df_Jul.iloc[0:1000]
print('MRR for Jul is',df_Jul_1000['Reciprocal'].mean())

df_Aug=df_final[df_final['Month']==8]
df_Aug=df_Aug.sort_values(by='Reciprocal Count',ascending=False)
df_Aug_1000=df_Aug.iloc[0:1000]
print('MRR for Aug is',df_Aug_1000['Reciprocal'].mean())

df_Sep=df_final[df_final['Month']==9]
df_Sep=df_Sep.sort_values(by='Reciprocal Count',ascending=False)
df_Sep_1000=df_Sep.iloc[0:1000]
print('MRR for Sep is',df_Sep_1000['Reciprocal'].mean())


df_Oct=df_final[df_final['Month']==10]
df_Oct=df_Oct.sort_values(by='Reciprocal Count',ascending=False)
df_Oct_1000=df_Oct.iloc[0:1000]
print('MRR for Oct is',df_Oct_1000['Reciprocal'].mean())


df_Nov=df_final[df_final['Month']==11]
df_Nov=df_Nov.sort_values(by='Reciprocal Count',ascending=False)
df_Nov_1000=df_Nov.iloc[0:1000]
print('MRR for Nov is',df_Nov_1000['Reciprocal'].mean())


df_Dec=df_final[df_final['Month']==12]
df_Dec=df_Dec.sort_values(by='Reciprocal Count',ascending=False)
df_Dec_1000=df_Dec.iloc[0:1000]
print('MRR for Dec is',df_Dec_1000['Reciprocal'].mean())

df_Jan=df_final[df_final['Month']==1]
df_Jan=df_Jan.sort_values(by='Reciprocal Count',ascending=False)
df_Jan_1000=df_Jan.iloc[0:1000]
print('MRR for Jan is',df_Jan_1000['Reciprocal'].mean())

### Laptop

In [ ]:
df=df_laptop
#create a Reciprocal rank column
df['Reciprocal']=1/df['Product Rank (evar12)']

#Add month column
df['Time Stamp (prop3)']=pd.to_datetime(df['Time Stamp (prop3)'])
df['Month']=df['Time Stamp (prop3)'].dt.month
df=df[df['Month']!=6]

#Group the df by Month+ST such that we get Mean of each search term for the month
df_month_mrr=df.groupby(by=['Month','Search Term (evar18)']).mean().reset_index()
df_month_mrr=df_month_mrr[['Month','Search Term (evar18)','Reciprocal']]

#Group the df by Month+ST such that we get count of each search term for the month
df_month_count=df.groupby(by=['Month','Search Term (evar18)']).count().reset_index()
df_month_count=df_month_count[['Search Term (evar18)','Reciprocal','Month']]
df_month_count.rename(columns={'Reciprocal':'Reciprocal Count'},inplace=True)

#Final dataframe
df_final=df_month_count.merge(df_month_mrr,on=['Month','Search Term (evar18)'])

#create df for each month, sort the df by the count of each sort term, get the first 1000 rows and find the mean of this df
df_Jul=df_final[df_final['Month']==7]
df_Jul=df_Jul.sort_values(by='Reciprocal Count',ascending=False)
df_Jul_1000=df_Jul.iloc[0:1000]
print('MRR for Jul is',df_Jul_1000['Reciprocal'].mean())

df_Aug=df_final[df_final['Month']==8]
df_Aug=df_Aug.sort_values(by='Reciprocal Count',ascending=False)
df_Aug_1000=df_Aug.iloc[0:1000]
print('MRR for Aug is',df_Aug_1000['Reciprocal'].mean())

df_Sep=df_final[df_final['Month']==9]
df_Sep=df_Sep.sort_values(by='Reciprocal Count',ascending=False)
df_Sep_1000=df_Sep.iloc[0:1000]
print('MRR for Sep is',df_Sep_1000['Reciprocal'].mean())


df_Oct=df_final[df_final['Month']==10]
df_Oct=df_Oct.sort_values(by='Reciprocal Count',ascending=False)
df_Oct_1000=df_Oct.iloc[0:1000]
print('MRR for Oct is',df_Oct_1000['Reciprocal'].mean())


df_Nov=df_final[df_final['Month']==11]
df_Nov=df_Nov.sort_values(by='Reciprocal Count',ascending=False)
df_Nov_1000=df_Nov.iloc[0:1000]
print('MRR for Nov is',df_Nov_1000['Reciprocal'].mean())


df_Dec=df_final[df_final['Month']==12]
df_Dec=df_Dec.sort_values(by='Reciprocal Count',ascending=False)
df_Dec_1000=df_Dec.iloc[0:1000]
print('MRR for Dec is',df_Dec_1000['Reciprocal'].mean())

df_Jan=df_final[df_final['Month']==1]
df_Jan=df_Jan.sort_values(by='Reciprocal Count',ascending=False)
df_Jan_1000=df_Jan.iloc[0:1000]
print('MRR for Jan is',df_Jan_1000['Reciprocal'].mean())

### Tablet

In [ ]:
df=df_tablet
#create a Reciprocal rank column
df['Reciprocal']=1/df['Product Rank (evar12)']

#Add month column
df['Time Stamp (prop3)']=pd.to_datetime(df['Time Stamp (prop3)'])
df['Month']=df['Time Stamp (prop3)'].dt.month
df=df[df['Month']!=6]

#Group the df by Month+ST such that we get Mean of each search term for the month
df_month_mrr=df.groupby(by=['Month','Search Term (evar18)']).mean().reset_index()
df_month_mrr=df_month_mrr[['Month','Search Term (evar18)','Reciprocal']]

#Group the df by Month+ST such that we get count of each search term for the month
df_month_count=df.groupby(by=['Month','Search Term (evar18)']).count().reset_index()
df_month_count=df_month_count[['Search Term (evar18)','Reciprocal','Month']]
df_month_count.rename(columns={'Reciprocal':'Reciprocal Count'},inplace=True)

#Final dataframe
df_final=df_month_count.merge(df_month_mrr,on=['Month','Search Term (evar18)'])

#create df for each month, sort the df by the count of each sort term, get the first 1000 rows and find the mean of this df
df_Jul=df_final[df_final['Month']==7]
df_Jul=df_Jul.sort_values(by='Reciprocal Count',ascending=False)
df_Jul_1000=df_Jul.iloc[0:1000]
print('MRR for Jul is',df_Jul_1000['Reciprocal'].mean())

df_Aug=df_final[df_final['Month']==8]
df_Aug=df_Aug.sort_values(by='Reciprocal Count',ascending=False)
df_Aug_1000=df_Aug.iloc[0:1000]
print('MRR for Aug is',df_Aug_1000['Reciprocal'].mean())

df_Sep=df_final[df_final['Month']==9]
df_Sep=df_Sep.sort_values(by='Reciprocal Count',ascending=False)
df_Sep_1000=df_Sep.iloc[0:1000]
print('MRR for Sep is',df_Sep_1000['Reciprocal'].mean())


df_Oct=df_final[df_final['Month']==10]
df_Oct=df_Oct.sort_values(by='Reciprocal Count',ascending=False)
df_Oct_1000=df_Oct.iloc[0:1000]
print('MRR for Oct is',df_Oct_1000['Reciprocal'].mean())


df_Nov=df_final[df_final['Month']==11]
df_Nov=df_Nov.sort_values(by='Reciprocal Count',ascending=False)
df_Nov_1000=df_Nov.iloc[0:1000]
print('MRR for Nov is',df_Nov_1000['Reciprocal'].mean())


df_Dec=df_final[df_final['Month']==12]
df_Dec=df_Dec.sort_values(by='Reciprocal Count',ascending=False)
df_Dec_1000=df_Dec.iloc[0:1000]
print('MRR for Dec is',df_Dec_1000['Reciprocal'].mean())

df_Jan=df_final[df_final['Month']==1]
df_Jan=df_Jan.sort_values(by='Reciprocal Count',ascending=False)
df_Jan_1000=df_Jan.iloc[0:1000]
print('MRR for Jan is',df_Jan_1000['Reciprocal'].mean())